# Data Reports

use pandas-profiling to generate data reports.  
Also adds W~Q correlations to metadata.csv

In [1]:
import os
from tqdm import tqdm
import json
import warnings
from camelsp import Bundesland, util
import pandas as pd

First load all NUTS folders found in the output dir

In [2]:
nuts = sorted([nt for nt in os.listdir(Bundesland('Hessen').base_path) if nt.startswith('DE')])
nuts

['DE1',
 'DE2',
 'DE4',
 'DE7',
 'DE8',
 'DE9',
 'DEA',
 'DEB',
 'DEC',
 'DED',
 'DEE',
 'DEF',
 'DEG']

Und jetzt gib ihm.

In [3]:
# set to true, if new output data was added (ie. rainfall)
REPLACE = False

# create for each report
for ID in nuts:
    with Bundesland(ID) as bl:
        with warnings.catch_warnings(record=True) as warn:
            nuts_ids = bl.nuts_table.nuts_id.values.tolist()

            for nuts_id in tqdm(nuts_ids, desc=ID):
                # write the report files
                try:
                    bl.generate_reports(nuts_ids=nuts_id, fmt='html', if_exists='replace' if REPLACE else 'omit')
                    bl.generate_reports(nuts_ids=nuts_id, fmt='json', if_exists='replace' if REPLACE else 'omit')
                except Exception as e:
                    print(str(e))
                    warnings.warn(str(e))

            if len(warn) > 0:
                print(f"FutureWarnings: {len([w for w in warn if w.category == FutureWarning])}")
                print(f"Other warnings: {len([w for w in warn if w.category != FutureWarning])} (possibly missing data files).")

DE1:   0%|          | 0/252 [00:00<?, ?it/s]

DEG: 100%|██████████| 63/63 [03:19<00:00,  3.16s/it]


In [3]:
# set to true, if new output data was added (ie. rainfall)
REPLACE = True

ID = 'DE9'

# create report for one bl
with Bundesland(ID) as bl:
    with warnings.catch_warnings(record=True) as warn:
        nuts_ids = bl.nuts_table.nuts_id.values.tolist()

        for nuts_id in tqdm(nuts_ids, desc=ID):
            # write the report files
            try:
                bl.generate_reports(nuts_ids=nuts_id, fmt='html', if_exists='replace' if REPLACE else 'omit')
                bl.generate_reports(nuts_ids=nuts_id, fmt='json', if_exists='replace' if REPLACE else 'omit')
            except Exception as e:
                print(str(e))
                warnings.warn(str(e))

        if len(warn) > 0:
            print(f"FutureWarnings: {len([w for w in warn if w.category == FutureWarning])}")
            print(f"Other warnings: {len([w for w in warn if w.category != FutureWarning])} (possibly missing data files).")

DE9: 100%|██████████| 294/294 [15:35<00:00,  3.18s/it]


## Add W ~ Q correlations to metadata

The data reports contain correlations between all data variables. We can extract the pearson's correlation coefficient or the spearman rank correlation and add to the preliminary metadata file

In [4]:
util.get_metadata()

,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,gauge_name,waterbody_name,gauge_elevation,area,x,y,lon,lat,q_count,w_count,q_w_pearson,q_w_spearman
0,DEG10000,573000,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,Ammern,Unstrut,210.243,182.700000,4.352221e+06,3.124617e+06,10.446993,51.231727,29646.0,29646.0,0.969240,0.976895
1,DEG10010,447000,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,Arenshausen,Leine,196.288,275.000000,4.318941e+06,3.140875e+06,9.970428,51.378709,22707.0,22707.0,0.709148,0.685209
2,DEG10020,574200,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,Arnstadt,Gera,293.577,174.700000,4.386764e+06,3.077926e+06,10.933022,50.809106,35490.0,35490.0,0.958767,0.962387
3,DEG10030,576500,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,Berga,Weiße Elster,218.995,1383.000000,4.473276e+06,3.073272e+06,12.157989,50.750857,12845.0,12845.0,0.502141,0.553354
4,DEG10040,570210,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,Blankenstein-Rosenthal,Saale,410.517,1013.000000,4.442190e+06,3.033884e+06,11.704738,50.404273,21246.0,21246.0,0.940139,0.977699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901,DE912890,5934145,./DE9/DE912890/DE912890_data.csv,DE9,Niedersachsen,Jeetzel UW,NaN,NaN,NaN,inf,inf,inf,inf,0.0,18689.0,NaN,NaN
2902,DE912900,5985101,./DE9/DE912900/DE912900_data.csv,DE9,Niedersachsen,Bremervörde,NaN,NaN,NaN,inf,inf,inf,inf,0.0,9923.0,NaN,NaN
2903,DE912910,5986107,./DE9/DE912910/DE912910_data.csv,DE9,Niedersachsen,Hollen,NaN,NaN,NaN,inf,inf,inf,inf,0.0,12845.0,NaN,NaN
2904,DE912920,9286164,./DE9/DE912920/DE912920_data.csv,DE9,Niedersachsen,Laar,NaN,NaN,1749.573210,4.100147e+06,3.283012e+06,6.738424,52.611652,0.0,19450.0,NaN,NaN


In [5]:
for NUTS in util._NUTS_LVL2_NAMES.keys():    
    # process this federal state
    with Bundesland(NUTS) as bl:    
        pearson = []
        spearman = []

        # get the metadata
        meta = bl.metadata

        # load the Data-report for each
        for camels_id in tqdm(meta.camels_id.values, desc=NUTS):
            p = os.path.join(bl.base_path, 'reports', f'{camels_id}.json')
            
            # check if the report exists
            if not os.path.exists(p):
                pearson.append(None)
                spearman.append(None)
                continue
            
            with open(p, 'r') as f:
                report = json.load(f)
            
            try:
                q = [o for o in report['correlations']['pearson'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['pearson'] if o['w'] == 1.0][0]
                pearson.append(q['w'])
            except:
                pearson.append(None)
            
            try:
                q = [o for o in report['correlations']['spearman'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['spearman'] if o['w'] == 1.0][0]
                spearman.append(q['w'])
            except:
                spearman.append(None)
        
        # all collected, return now
        corrs = pd.DataFrame({'camels_id': meta.camels_id.values, 'q_w_pearson': pearson, 'q_w_spearman': spearman})

        # update
        bl.update_metadata(corrs)

metadata = util.get_metadata()
metadata[metadata['nuts_lvl2'] == 'DE9']

DE1:   0%|          | 0/252 [00:00<?, ?it/s]

DE2: 100%|██████████| 535/535 [00:08<00:00, 62.82it/s]
DE3: 0it [00:00, ?it/s]
DE4: 100%|██████████| 233/233 [00:02<00:00, 84.48it/s] 
DE5: 0it [00:00, ?it/s]
DE6: 0it [00:00, ?it/s]
DEG: 100%|██████████| 63/63 [00:01<00:00, 58.48it/s]


,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,gauge_name,waterbody_name,gauge_elevation,area,x,y,lon,lat,q_count,w_count,q_w_pearson,q_w_spearman
2612,DE910000,3183101,./DE9/DE910000/DE910000_data.csv,DE9,Niedersachsen,Sudendorf,NaN,NaN,121.560123,4.183280e+06,3.217135e+06,7.992302,52.047079,12845.0,12845.0,0.921013,0.928097
2613,DE910010,3346103,./DE9/DE910010/DE910010_data.csv,DE9,Niedersachsen,Schwege,NaN,NaN,47.371025,4.178458e+06,3.218442e+06,7.921498,52.057597,12845.0,12845.0,0.828968,0.816012
2614,DE910020,3437108,./DE9/DE910020/DE910020_data.csv,DE9,Niedersachsen,Beesten,NaN,NaN,407.147698,4.150721e+06,3.261657e+06,7.495577,52.437915,5966.0,5966.0,0.850115,0.637185
2615,DE910030,3445100,./DE9/DE910030/DE910030_data.csv,DE9,Niedersachsen,Spelle,NaN,NaN,NaN,inf,inf,inf,inf,6789.0,6789.0,0.836398,0.756127
2616,DE910040,3449100,./DE9/DE910040/DE910040_data.csv,DE9,Niedersachsen,Spelle,NaN,NaN,149.735031,4.155177e+06,3.254250e+06,7.564736,52.372753,4597.0,4597.0,0.824894,0.715392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901,DE912890,5934145,./DE9/DE912890/DE912890_data.csv,DE9,Niedersachsen,Jeetzel UW,NaN,NaN,NaN,inf,inf,inf,inf,0.0,18689.0,NaN,NaN
2902,DE912900,5985101,./DE9/DE912900/DE912900_data.csv,DE9,Niedersachsen,Bremervörde,NaN,NaN,NaN,inf,inf,inf,inf,0.0,9923.0,NaN,NaN
2903,DE912910,5986107,./DE9/DE912910/DE912910_data.csv,DE9,Niedersachsen,Hollen,NaN,NaN,NaN,inf,inf,inf,inf,0.0,12845.0,NaN,NaN
2904,DE912920,9286164,./DE9/DE912920/DE912920_data.csv,DE9,Niedersachsen,Laar,NaN,NaN,1749.573210,4.100147e+06,3.283012e+06,6.738424,52.611652,0.0,19450.0,NaN,NaN
